In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from keras.callbacks import ReduceLROnPlateau,EarlyStopping
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import functional as F

In [2]:
batch_size = 100
img_height = 300
img_width = 300


In [3]:
train_set = tf.keras.preprocessing.image_dataset_from_directory(
   "C:\\Users\\sarthak mohapatra\\Downloads\\traffic dataset\\train",
    image_size= (img_height, img_width),
    batch_size=batch_size

)

test_set = tf.keras.preprocessing.image_dataset_from_directory(
    "C:\\Users\\sarthak mohapatra\\Downloads\\traffic dataset\\test",
    image_size= (img_height, img_width),
    batch_size=batch_size)

val_set =  tf.keras.preprocessing.image_dataset_from_directory(
 "C:\\Users\\sarthak mohapatra\\Downloads\\traffic dataset\\valid",

    image_size= (img_height, img_width),
    batch_size=batch_size)

Found 3130 files belonging to 2 classes.
Found 279 files belonging to 2 classes.
Found 547 files belonging to 2 classes.


In [4]:
class_names = train_set.class_names

print(class_names)

['images', 'no traffic']


In [5]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train = train_set.cache().prefetch(buffer_size=AUTOTUNE)
test = test_set.cache().prefetch(buffer_size=AUTOTUNE)

img_shape = (img_height, img_width, 3)

In [16]:
base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape,include_top=False, weights='imagenet')

base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512,activation='relu'),
    layers.Dense(2, activation= 'softmax')
])

In [17]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])
lr=ReduceLROnPlateau(monitor='val_accuracy',patience=2,verbse=0.5,factor=0.5,min_lr=0.0001)
early_stop=EarlyStopping(monitor='val_loss',patience=2,restore_best_weights=True)

In [19]:
history = model.fit(train, validation_data = val_set, epochs = 20,callbacks=[lr])

Epoch 1/20


32/32 [==============================] - 86s 3s/step - loss: 0.5775 - accuracy: 0.6933 - val_loss: 0.7197 - val_accuracy: 0.5375 - lr: 0.0010
Epoch 2/20
32/32 [==============================] - 79s 2s/step - loss: 0.4421 - accuracy: 0.7869 - val_loss: 0.8291 - val_accuracy: 0.5503 - lr: 0.0010
Epoch 3/20
32/32 [==============================] - 79s 2s/step - loss: 0.3233 - accuracy: 0.8559 - val_loss: 0.8153 - val_accuracy: 0.5960 - lr: 0.0010
Epoch 4/20
32/32 [==============================] - 79s 2s/step - loss: 0.2423 - accuracy: 0.9016 - val_loss: 1.5051 - val_accuracy: 0.5229 - lr: 0.0010
Epoch 5/20
32/32 [==============================] - 76s 2s/step - loss: 0.2512 - accuracy: 0.8824 - val_loss: 0.8311 - val_accuracy: 0.6234 - lr: 0.0010
Epoch 6/20
32/32 [==============================] - 77s 2s/step - loss: 0.1905 - accuracy: 0.9198 - val_loss: 0.9496 - val_accuracy: 0.6234 - lr: 0.0010
Epoch 7/20
32/32 [==============================] - 77s 2s/step - loss: 0.1930 - accuracy: 0.

KeyboardInterrupt: 

In [ ]:
def predict_frame(img):
    img_array = tf.keras.utils.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    prediction=(model.predict(img_batch) > 0.5).astype("int32")
    if(prediction[0][0]==0):
        return("no-traffic")
    else:
        return("traffic")

In [ ]:
label=[]
c=1
cap=cv2.VideoCapture(r"C:\Users\sarthak mohapatra\Downloads\15 minutes of heavy traffic noise in India _ 14-08-2022.mp4")
while True:
    ret, frame = cap.read()
    if c%30==0:
     print(c)
    #ret, frame = cap.read()
     resize_frame=cv2.resize(frame,(300,300))
  
     predict=predict_frame(resize_frame)
     label.append(predict)
    c+=1
    cv2.imshow('frame',frame)
    key=cv2.waitKey(1)&0xFF
    if  key == ord("a"):
        break
cap.release()
cv2.destroyAllWindows()

30
1/1 [==============================] - 1s 907ms/step
60
1/1 [==============================] - 0s 34ms/step
90
1/1 [==============================] - 0s 32ms/step
120
1/1 [==============================] - 0s 33ms/step
150
1/1 [==============================] - 0s 34ms/step
180
1/1 [==============================] - 0s 31ms/step
210
1/1 [==============================] - 0s 33ms/step
240
1/1 [==============================] - 0s 32ms/step
270
1/1 [==============================] - 0s 41ms/step
300
1/1 [==============================] - 0s 34ms/step
330
1/1 [==============================] - 0s 34ms/step
360
1/1 [==============================] - 0s 36ms/step
390
1/1 [==============================] - 0s 33ms/step
420
1/1 [==============================] - 0s 34ms/step
450
1/1 [==============================] - 0s 32ms/step
480
1/1 [==============================] - 0s 34ms/step
510
1/1 [==============================] - 0s 42ms/step
540
1/1 [==============================] - 0s 37ms

In [ ]:
print(label[30])

traffic
